# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert words to vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>
<hr>

_Embeddings_ are representation of words thanks to vectors. These embeddings can be learnt within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representation into an RNN. 



# The data

Keras provides many datasets, among which the ÌMDB dataset: it corresponds to sentences that are movie reviews. Each of them is related to a score given by the review writer.

❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that a too large number of sentences will make your compute slow down, or even freeze - your RAM can even overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [1]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2022-05-27 14:04:20.443328: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-27 14:04:20.443463: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


2022-05-27 14:04:27.581390: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-27 14:04:27.581575: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-27 14:04:27.581717: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-3PI3NFG): /proc/driver/nvidia/version does not exist
2022-05-27 14:04:27.582511: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In the previous exercise, we jointly learnt a representation for the words, and feed this representation to a RNN, as shown here : 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which can slow the convergence, and make it harder!

For that reason, we will separate the steps of learning the word representation and feeding it to a RNN. As shown here : 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with the word2vec.

The drawback is indeed that the learnt embedding is not _specifically_ designed for our task. However, learning it independently of the task at hand (sentiment analysis) has some advantages : 
- it is very fast to do in general (with word2vec)
- the representation learnt by word2vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with word2vec and see how meaningful it is!


# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 'dog' --> [0.1, -0.3, 0.8]
- 'cat' --> [-1.1, 2.3, 0.7]
- 'apple' --> [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

What you expect is to have representation such as words with close meanings are close in this embedding space. As on this example:

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! The following code imports word2vec from [GENSIM](https://radimrehurek.com/gensim/), a great python package that makes the use of the word2vec algorithm easy, fast and accurate - which is not an easy task. The second line learns the embedding representation of the words thanks to the sentences in `X_train`.

In [2]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)

Let's look at the embedded representation of some words.

You can use `word2vec.wv` as a dictionary.
For instance, `word2vec.wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation were not learn). 

In [4]:
word2vec.wv['chess']

KeyError: "Key 'chess' not present"

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [8]:
len(word2vec.wv['god']), word2vec.wv['god']

(100,
 array([-0.03680298,  0.06401711, -0.08397809,  0.2641723 , -0.01449071,
        -0.6396183 ,  0.16820778,  0.5873453 , -0.3893225 , -0.21072736,
        -0.19760266, -0.41165712,  0.1612972 ,  0.10416951, -0.07214358,
        -0.5061737 ,  0.4117711 , -0.5051072 , -0.05427171, -0.7356505 ,
         0.23389526,  0.21239302,  0.2976067 ,  0.01359967, -0.13407095,
        -0.1036624 , -0.4286783 , -0.14694037, -0.14323397,  0.06252497,
         0.6057078 ,  0.11274226,  0.14449757, -0.26090586, -0.31850266,
         0.60661304,  0.08323491, -0.02152824, -0.23736598, -0.69261545,
        -0.18316965, -0.28469762,  0.09388344,  0.22384465,  0.10480733,
        -0.06107022, -0.23790903, -0.18577379,  0.146531  ,  0.05455402,
         0.01273118, -0.29605842, -0.03694272, -0.16657822, -0.06624108,
         0.16522874,  0.14743084, -0.16933775, -0.4832319 , -0.08573813,
         0.05881061, -0.06508325,  0.2884791 ,  0.02632209, -0.6455664 ,
         0.3380911 ,  0.08440886,  0.20079014

How to know if this embedding make any sense? To do that, we will check that words with a close meaning have close representations. 

Let's use the `word2vec.most_similar(...)` method that, given an input word, display the "closest" words in the embedding space. If the embedding is well done, then words that have close meanings will have close representation in the embedding space.

❓ **Question** ❓ Test the `most_similar` method on different words. 

❗ **Remark** ❗ Indeed, the quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [12]:
word2vec.wv.most_similar('god', topn=10)

[('curious', 0.9694924354553223),
 ('saying', 0.963270902633667),
 ('seriously', 0.9614155292510986),
 ('happy', 0.9538231492042542),
 ('sick', 0.9534871578216553),
 ('hey', 0.9531416296958923),
 ('ignore', 0.9531086087226868),
 ('complaining', 0.9524694085121155),
 ('impression', 0.9509376883506775),
 ('stopped', 0.9496915936470032)]

Similarly to `most_similar` used on words directly, we can use `similar_by_vector` on vectors to do the same thing :

In [13]:
word2vec.wv.similar_by_vector(word2vec.wv['god'], topn=10)

[('god', 1.0000001192092896),
 ('curious', 0.9694924354553223),
 ('saying', 0.963270902633667),
 ('seriously', 0.9614155292510986),
 ('happy', 0.9538232684135437),
 ('sick', 0.9534871578216553),
 ('hey', 0.9531416296958923),
 ('ignore', 0.9531086087226868),
 ('complaining', 0.9524694085121155),
 ('impression', 0.9509376883506775)]

# Arithmetic on words

Now, let's do mathematical operations on words - meaning on their vector representations!

As any word is represented as a vector, you can do basic arithmetic as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [14]:
word2vec.wv['good']-word2vec.wv['bad']

array([-0.28005618, -0.18747345,  0.30082   ,  0.06856149, -0.03558493,
       -0.0110743 , -0.32385606, -0.12167835,  0.14893627, -0.28300148,
        0.08375421,  0.14183456,  0.3073163 , -0.10949078, -0.17642698,
        0.48008737, -0.41206312,  0.17303428, -0.20498577,  0.35489595,
       -0.13855958,  0.08357207,  0.20414066,  0.10536931,  0.13879836,
       -0.01152931,  0.0989508 ,  0.47291252, -0.12381962, -0.25394672,
       -0.2485261 ,  0.37924883, -0.05525398,  0.16823022, -0.23257756,
       -0.30658734,  0.20420027, -0.24093246, -0.42545122,  0.8709073 ,
       -0.39531928, -0.09926367, -0.5724615 , -0.01662689,  0.46459502,
       -0.36369362,  0.45914692, -0.33182955, -0.02669966,  0.43783072,
        0.22712949, -0.15075284, -0.46627662,  0.05530165,  0.00538534,
        0.31877068,  0.03338394,  0.14075002,  0.24821508,  0.5898334 ,
        0.18960804,  0.22482586, -0.26835954, -0.43921453,  0.10189456,
       -0.02515483, -0.50131077, -0.29080018,  0.58867335,  0.14

Now, image for a second that, somehow, the following equality holds true - just for a second

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to 

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be foolish of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [16]:
res=word2vec.wv['good']-word2vec.wv['bad']+word2vec.wv['stupid']
res

array([ 0.10187221, -0.16735074,  0.24902636, -0.13465735,  0.17229861,
       -0.62857056, -0.47174713,  0.38289386, -0.37573683, -0.57595253,
       -0.04074829, -0.30943602,  0.17655675,  0.2150847 ,  0.07806224,
        0.08177847,  0.00677758, -0.23503044, -0.4762532 , -0.53286254,
        0.36132586,  0.20745789,  1.1448773 , -0.07159847, -0.1395405 ,
       -0.128706  , -0.12623465,  0.28653967, -0.33447704, -0.22492026,
        0.46825403,  0.43250436,  0.12778333, -0.12253393, -0.6809865 ,
        0.30872446,  0.24193859, -0.16065542, -0.49462628,  0.07009423,
       -0.25793403, -0.6962443 , -0.250085  ,  0.5011687 ,  0.61848176,
       -0.1533505 ,  0.05483153, -0.7101619 , -0.30439818,  0.63198674,
        0.30162948, -0.3786596 , -0.6179149 , -0.00501607, -0.02150378,
        0.54259604,  0.20966369,  0.22598651, -0.29253596,  0.15170678,
        0.16936353, -0.08474818,  0.30417985, -0.238076  , -0.45111758,
        0.52263564, -0.25428736, -0.07111889,  0.24647546, -0.00

We earlier said that, for any vector, it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vector (thanks to the `word2vec.wv.similar_by_vector` function) of `res`

In [17]:
word2vec.wv.similar_by_vector(res, topn=2)[1]

('nice', 0.8270463943481445)

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [53]:
'bathroom' in word2vec2.wv.key_to_index.keys()

True

You might wonder where does this magic comes from (at quite a low price, you just run a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)` , actually trains a internal neural network (that you don't see).  

In a nutshell, this internal NN predicts a word from the surroundings words in a sentences. So it chooses many splits in the different sentences, choose some words as inputs $X$  and a word as output $y$ which it tries to predict, in the embedding space.

And as any neural network, Word2Vec has some hyperparameters. Let's check some. 


# Word2Vec hyperparameters


❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the corresponding embedding is of your selected size.

In [19]:
word2vec2 = Word2Vec(sentences=X_train, vector_size=200)
len(word2vec2.wv['god'])

200

❓ **Question** ❓ Use the `word2vec.wv.key_to_index` attribute to display the size of the learnt vocabulary. On the other hand, compare it to the number of different words in `X_train`.

In [26]:
len(word2vec2.wv.key_to_index.keys()), vocab_size

(8006, 30419)

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab_size=len(tokenizer.word_index)

There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been train on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec :  `min_count`. 

`min_count` is a integer that tells you how many occurences a given word should have to be learn in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to have only words that are sufficiently present in the corpus to have a robust embedded representation

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vec that you have trained (you can choose any `vector_size` you want).

In [35]:
word2vec3=Word2Vec(sentences=X_train, vector_size=200, min_count=10)
len(word2vec3.wv.key_to_index.keys()), vocab_size

(4503, 30419)

In [36]:
word2vec4=Word2Vec(sentences=X_train, vector_size=200, min_count=1)
len(word2vec4.wv.key_to_index.keys()), vocab_size

(30419, 30419)

Remember that we say that word2vec has an internal neural network that it optimizes based on some predictions? These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the word2vec with different `window` sizes.

❓ **Question** ❓ Learn a new `word2vec_5` model with a `window` different than previously (default is 5).

In [37]:
word2vec5=Word2Vec(sentences=X_train, vector_size=200, min_count=5, window=10)
len(word2vec5.wv.key_to_index.keys()), vocab_size

(8006, 30419)

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the one that you should start changing to get a better performance for your model.

But you can also look at other arguments in the [documentation](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)



# Convert our train and test set to RNN ready data

Remember that word2vec is the first step to the overall process of feeding such a representation into a RNN, as shown here :

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be feed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (be sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- for the test set, this is understandable: some words were not in the train set and thus their embedded representation is unknown
- for the train set, due to `min_count` hyperparameter, not all the words have a vector representation

In any case, just skip the missing words here.

In [42]:
np.array(word2vec.wv["this"].tolist())

array([ 0.77012706, -0.11993534,  0.35350898,  0.81576592, -0.15466654,
       -1.26268017, -0.32967833,  1.30717397, -0.75693929, -1.03244388,
        0.8256135 , -0.22497889,  0.3305831 ,  0.22459954,  0.11601797,
       -0.1107952 ,  0.59392142, -0.52114409, -0.50502896, -1.89296687,
       -0.06058493,  0.43620923,  1.80419719,  0.60057127,  0.06445289,
        0.03877936, -0.83228439, -0.13954824,  0.16902114,  0.09424441,
        0.02896009,  0.51776415,  1.47067618,  0.99676251, -0.2187891 ,
        1.15909338, -0.01173606, -0.09742951, -0.72023803, -0.48836759,
       -0.54429901,  0.59727836, -0.77040309,  0.28589845, -1.52991986,
        0.401997  , -0.43425798, -0.09005369, -1.22441673,  0.45350099,
       -0.85628796, -0.0591313 , -0.97512686, -0.23849922,  1.47186828,
       -0.89138889, -0.72320622, -0.73164767, -0.40086731,  0.25904191,
       -1.16351175, -0.11598156,  0.82558316, -0.79637957, -1.92107368,
        0.31870165, -0.52684367, -1.17201555, -0.33757642,  0.17

In [50]:
output=[]
example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
for x in example:
    output.append(x)
np.array(output)

array(['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever'],
      dtype='<U6')

In [54]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
    output=[]
    for x in example:
        if x in word2vec2.wv.key_to_index.keys():
            output.append(word2vec.wv[x])
    return np.array(output)
    
### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

AssertionError: 

❓ **Question** ❓ Write a function that, given a list of sentence (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

Hint: Use the previous function `embed_sentence`

In [56]:
def embedding(word2vec, sentences):
    output=[]
    for sentence in sentences:
        output.append(embed_sentence(word2vec, sentence))
    return output
        
X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad them in order to have tensors that can be divided in batch sizes during the optimization. Store the padedd values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [57]:
### YOUR CODE HERE

assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)

NameError: name 'X_train_pad' is not defined